<a href="https://colab.research.google.com/github/lguloy/House-Pricing-Predictions/blob/Housing_predictions_Clayton/Dummies_House_Price_Train_Test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
# Find the latest version of spark 3.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
spark_version = 'spark-3.0.3'
# spark_version = 'spark-3.<enter version>'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Get:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Hit:9 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:11 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Ign:12 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Packages
Get:12 https://developer.downlo

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CloudETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [5]:

from pyspark import SparkFiles
# Load in train.csv from S3 into a DataFrame
url = "https://finalprojectbootcamp.s3.us-east-2.amazonaws.com/train_converted_df.csv"
spark.sparkContext.addFile(url)

df = spark.read.option('header', 'true').csv(SparkFiles.get("train_converted_df.csv"), inferSchema=True, sep=',', timestampFormat="mm/dd/yy")
df.show(10)

#Convert to Pandas to allow train_test_split usage
df = df.select("*").toPandas()
#df = df.dropna(how="any")

+---+----------+-----------+-------+-----------+-----------+---------+------------+----------+---------+---------+--------+--------+------------+------------+----------+------------+----------+---------+-----------+---------+------------+---------+------------+------------+--------+--------+------------+------------+-----------+------------+----------+----------+-----------+-----------+------------+----------+----------+----------+----------+----------+-----------+-------------+-----------+--------+------+-----+-------+------+------+---------+-----------+-----------+-----------+-----------+----------+------------+------------+------------+---------------+---------------+---------------+----------------+-----------------+-------------+-------------+----------------+-------------+-------------+--------------------+-------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-------------------+----------

In [7]:
df.head(50)

,Id,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,ExterQual,ExterCond,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,HeatingQC,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,...,Foundation_PConc,Foundation_Slab,Foundation_Stone,Foundation_Wood,Heating_GasA,Heating_GasW,Heating_Grav,Heating_OthW,Heating_Wall,CentralAir_Y,Electrical_FuseF,Electrical_FuseP,Electrical_Mix,Electrical_SBrkr,GarageType_Attchd,GarageType_Basment,GarageType_BuiltIn,GarageType_CarPort,GarageType_Detchd,PavedDrive_P,PavedDrive_Y,MiscFeature_Othr,MiscFeature_Shed,MiscFeature_TenC,SaleType_CWD,SaleType_Con,SaleType_ConLD,SaleType_ConLI,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial,firstflrsf,secondflrsf,thirdPorch
0,1,60,65.0,8450,7,5,2003,2003,196.0,4,3,4.0,3.0,1.0,6.0,706,1.0,0,150,856,5,0,1710,1,0,2,1,3,1,4,8,8,0,NaN,2003.0,2.0,2,548,3.0,3.0,...,1,0,0,0,1,0,0,0,0,1,0,0,0,1,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,856,854,0
1,2,20,80.0,9600,6,8,1976,1976,0.0,3,3,4.0,3.0,4.0,5.0,978,1.0,0,284,1262,5,0,1262,0,1,2,0,3,1,3,6,8,1,3.0,1976.0,2.0,2,460,3.0,3.0,...,0,0,0,0,1,0,0,0,0,1,0,0,0,1,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,1262,0,0
2,3,60,68.0,11250,7,5,2001,2002,162.0,4,3,4.0,3.0,2.0,6.0,486,1.0,0,434,920,5,0,1786,1,0,2,1,3,1,4,6,8,1,3.0,2001.0,2.0,2,608,3.0,3.0,...,1,0,0,0,1,0,0,0,0,1,0,0,0,1,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,920,866,0
3,4,70,60.0,9550,7,5,1915,1970,0.0,3,3,3.0,4.0,1.0,5.0,216,1.0,0,540,756,4,0,1717,1,0,1,0,3,1,4,7,8,1,4.0,1998.0,1.0,3,642,3.0,3.0,...,0,0,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,961,756,0
4,5,60,84.0,14260,8,5,2000,2000,350.0,4,3,4.0,3.0,3.0,6.0,655,1.0,0,490,1145,5,0,2198,1,0,2,1,4,1,4,9,8,1,3.0,2000.0,2.0,3,836,3.0,3.0,...,1,0,0,0,1,0,0,0,0,1,0,0,0,1,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,1145,1053,0
5,6,50,85.0,14115,5,5,1993,1995,0.0,3,3,4.0,3.0,1.0,6.0,732,1.0,0,64,796,5,0,1362,1,0,1,1,1,1,3,5,8,0,NaN,1993.0,1.0,2,480,3.0,3.0,...,0,0,0,1,1,0,0,0,0,1,0,0,0,1,1,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,796,566,320
6,7,20,75.0,10084,8,5,2004,2005,186.0,4,3,5.0,3.0,3.0,6.0,1369,1.0,0,317,1686,5,0,1694,1,0,2,0,3,1,4,7,8,1,4.0,2004.0,2.0,2,636,3.0,3.0,...,1,0,0,0,1,0,0,0,0,1,0,0,0,1,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,1694,0,0
7,8,60,NaN,10382,7,6,1973,1973,240.0,3,3,4.0,3.0,2.0,5.0,859,4.0,32,216,1107,5,0,2090,1,0,2,1,3,1,3,7,8,2,3.0,1973.0,2.0,2,484,3.0,3.0,...,0,0,0,0,1,0,0,0,0,1,0,0,0,1,1,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,1107,983,0
8,9,50,51.0,6120,7,5,1931,1950,0.0,3,3,3.0,3.0,1.0,1.0,0,1.0,0,952,952,4,0,1774,0,0,2,0,2,2,3,8,7,2,3.0,1931.0,1.0,2,468,2.0,3.0,...,0,0,0,0,1,0,0,0,0,1,1,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1022,752,0
9,10,190,50.0,7420,5,6,1939,1950,0.0,3,3,3.0,3.0,1.0,6.0,851,1.0,0,140,991,5,0,1077,1,0,1,0,2,2,3,5,8,2,3.0,1939.0,2.0,1,205,4.0,3.0,...,0,0,0,0,1,0,0,0,0,1,0,0,0,1,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,1077,0,0


In [ ]:
#Get the Feature names
import pandas as pd

features = [col for col in df.columns]
print(features)

print(type(features))

['Id', 'MSSubClass', 'LotFrontage', 'LotArea', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd', 'MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'TotRmsAbvGrd', 'Fireplaces', 'GarageYrBlt', 'GarageCars', 'GarageArea', 'WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea', 'MiscVal', 'MoSold', 'YrSold', 'SalePrice', 'MSZoning_FV', 'MSZoning_RH', 'MSZoning_RL', 'MSZoning_RM', 'Street_Pave', 'Alley_Pave', 'LotShape_IR2', 'LotShape_IR3', 'LotShape_Reg', 'LandContour_HLS', 'LandContour_Low', 'LandContour_Lvl', 'Utilities_NoSeWa', 'LotConfig_CulDSac', 'LotConfig_FR2', 'LotConfig_FR3', 'LotConfig_Inside', 'LandSlope_Mod', 'LandSlope_Sev', 'Neighborhood_Blueste', 'Neighborhood_BrDale', 'Neighborhood_BrkSide', 'Neighborhood_ClearCr', 'Neighborhood_CollgCr', 'Neighborhood_Crawfor', 'Neighborhood_

In [ ]:
#Import sklearn tools and dependencies
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

In [ ]:
# Assign the data to X and y

X1 = df[["1stFlrSF", "GrLivArea", "LotArea", "GarageArea", "BsmtUnfSF", "TotalBsmtSF", "LotFrontage", "GarageYrBlt", "MoSold", "YearBuilt"]]
y1 = df["SalePrice"]
print(X1.shape, y1.shape)

(1121, 10) (1121,)


In [ ]:
# Use train_test_split to create training and testing data

### BEGIN SOLUTION
from sklearn.model_selection import train_test_split

X1_train, X1_test, y1_train, y1_test = train_test_split(X1, y1, random_state=20, test_size=0.20)

### END SOLUTION

In [ ]:
# Create the model using RandomForestRegressor

### BEGIN SOLUTION
from sklearn.ensemble import RandomForestRegressor
model_1 = RandomForestRegressor(random_state=20)
### END SOLUTION

In [ ]:
# Fit the model to the training data and calculate the scores for the training and testing data

### BEGIN SOLUTION
model_1.fit(X1_train, y1_train)
training_score_1 = model_1.score(X1_train, y1_train)
testing_score_1 = model_1.score(X1_test, y1_test)

### END SOLUTION 

print(f"Training Score: {training_score_1}")
print(f"Testing Score: {testing_score_1}")

Training Score: 0.9705759221383611
Testing Score: 0.8219258498995223


In [ ]:
# Assign the data to X and y

X2 = df[["1stFlrSF", 
        "GrLivArea", 
        "LotArea", 
        "GarageArea", 
        "BsmtUnfSF", 
        "TotalBsmtSF", 
        "LotFrontage", 
        "GarageYrBlt", 
        "MoSold", 
        "YearBuilt",
        "YearRemodAdd",
        "BsmtFinSF1",
        "OpenPorchSF",
        "WoodDeckSF",
        "TotRmsAbvGrd",
        "YrSold",
        "MasVnrArea",
        "2ndFlrSF",
        "OverallQual",
        "BedroomAbvGr"
        ]]
y2 = df["SalePrice"]
print(X2.shape, y2.shape)

(1121, 20) (1121,)


In [ ]:
# Use train_test_split to create training and testing data

### BEGIN SOLUTION
from sklearn.model_selection import train_test_split

X2_train, X2_test, y2_train, y2_test = train_test_split(X2, y2, random_state=20, test_size=0.20)

### END SOLUTION

In [ ]:
# Create the model using RandomForestRegressor

### BEGIN SOLUTION
from sklearn.ensemble import RandomForestRegressor
model_2 = RandomForestRegressor(random_state=20)
### END SOLUTION

In [ ]:
# Fit the model to the training data and calculate the scores for the training and testing data

### BEGIN SOLUTION
model_2.fit(X2_train, y2_train)
training_score_2 = model_2.score(X2_train, y2_train)
testing_score_2 = model_2.score(X2_test, y2_test)

### END SOLUTION 

print(f"Training Score: {training_score_2}")
print(f"Testing Score: {testing_score_2}")

Training Score: 0.9768133124121899
Testing Score: 0.8807235473601069


In [ ]:
x_extra = df.drop("SalePrice", axis=1)
x_extra.corr()

,Id,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,TotRmsAbvGrd,Fireplaces,GarageYrBlt,GarageCars,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,MSZoning_FV,MSZoning_RH,MSZoning_RL,...,BsmtFinType1_LwQ,BsmtFinType1_Rec,BsmtFinType1_Unf,BsmtFinType2_BLQ,BsmtFinType2_GLQ,BsmtFinType2_LwQ,BsmtFinType2_Rec,BsmtFinType2_Unf,HeatingQC_Fa,HeatingQC_Gd,HeatingQC_Po,HeatingQC_TA,KitchenQual_Fa,KitchenQual_Gd,KitchenQual_TA,Functional_Maj2,Functional_Min1,Functional_Min2,Functional_Mod,Functional_Sev,Functional_Typ,FireplaceQu_Fa,FireplaceQu_Gd,FireplaceQu_Po,FireplaceQu_TA,GarageFinish_RFn,GarageFinish_Unf,GarageQual_Fa,GarageQual_Gd,GarageQual_Po,GarageQual_TA,GarageCond_Fa,GarageCond_Gd,GarageCond_Po,GarageCond_TA,PoolQC_Fa,PoolQC_Gd,Fence_GdWo,Fence_MnPrv,Fence_MnWw
Id,1.000000,0.021937,-0.013289,-0.040711,-0.058269,0.004387,-0.020862,-0.027664,-0.073472,-0.013751,0.012544,-0.012985,-0.023129,-0.008046,-0.002346,-0.039933,-0.011068,0.026113,-0.026774,0.007220,-0.010409,0.039831,0.025913,0.020012,-0.018273,-0.002039,-0.008125,-0.025889,-0.025060,-0.001972,0.009935,-0.066833,0.015183,0.048010,0.045799,-0.000570,0.013407,-0.021242,0.004998,0.044854,...,-0.035149,-0.008120,0.006417,-0.007915,-0.023230,0.014153,0.044804,0.004171,0.006134,-0.021902,-0.028478,0.019314,0.016568,0.014702,-0.006211,-0.015095,-0.026467,0.023858,-0.003156,NaN,0.000093,-0.017836,-0.010158,0.013651,0.010483,0.005408,-0.023388,0.011871,-0.008785,-0.041857,0.011807,0.017463,0.010370,-0.032181,-0.001795,0.037409,0.051080,0.025894,-0.008140,0.008382
MSSubClass,0.021937,1.000000,-0.386940,-0.198096,0.029522,-0.087859,0.025800,0.006645,0.040240,-0.070389,-0.075439,-0.145582,-0.247781,-0.252249,0.319328,0.024704,0.083365,-0.014681,0.012310,0.131278,0.203971,-0.032971,0.266012,0.047209,-0.031122,0.054701,-0.027411,-0.092607,-0.017988,0.004054,-0.017790,-0.039739,-0.021789,0.003166,-0.040689,-0.027170,-0.012448,0.170426,0.034887,-0.331023,...,-0.019044,-0.079952,0.026825,-0.039594,0.017905,-0.058365,-0.030783,0.063200,0.005376,-0.023911,-0.007977,0.064586,-0.007735,0.009403,0.011881,-0.033890,-0.030629,-0.003521,-0.007008,NaN,-0.008318,-0.008489,-0.033628,0.028955,0.010774,-0.040830,0.044277,-0.024527,0.079629,0.039932,-0.017884,0.008123,-0.005044,0.041908,-0.013977,-0.016348,0.014027,-0.053596,-0.062866,-0.028735
LotFrontage,-0.013289,-0.386940,1.000000,0.421184,0.241322,-0.046312,0.109726,0.086414,0.189969,0.241352,0.049305,0.115306,0.387620,0.451085,0.075004,0.011148,0.396306,0.118088,0.000434,0.185785,0.045678,0.270404,-0.003546,0.348421,0.260321,0.069878,0.286587,0.356851,0.082166,0.161815,0.014261,0.069716,0.035906,0.211746,0.001471,0.018815,0.013267,-0.106475,-0.049148,0.349723,...,-0.055754,0.023620,-0.047057,0.004075,-0.022510,0.008608,0.036514,-0.018759,-0.039187,-0.026108,-0.025458,-0.072199,-0.011040,0.042618,-0.135232,0.005143,-0.026925,0.002614,-0.000352,NaN,0.035779,-0.007361,0.146068,-0.019117,0.103679,0.132800,-0.243414,-0.064816,-0.015417,-0.019930,0.063863,-0.050418,-0.014530,-0.020158,0.055260,0.014527,0.215835,0.034729,-0.002172,-0.004950
LotArea,-0.040711,-0.198096,0.421184,1.000000,0.167525,-0.034348,0.029205,0.026848,0.106115,0.230441,0.138234,0.011288,0.302554,0.329679,0.074612,0.020039,0.307164,0.179052,-0.014282,0.129073,0.045183,0.137269,-0.018942,0.237918,0.255755,0.013731,0.172428,0.211362,0.133576,0.099170,-0.023631,0.012520,0.072517,0.109147,0.012790,0.008998,-0.006904,-0.088293,-0.040838,0.212576,...,-0.024622,-0.006258,-0.035027,-0.020176,0.024173,-0.001552,0.130219,-0.065901,-0.010133,-0.023005,-0.018839,-0.001397,-0.015641,-0.011495,-0.048895,-0.000524,-0.017178,0.003344,0.082782,NaN,-0.010214,-0.018898,0.138416,-0.026511,0.069839,0.046692,-0.108660,-0.022837,-0.006103,-0.003540,0.018986,-0

In [ ]:
x_extra.info(verbose=True, null_counts=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1121 entries, 0 to 1459
Data columns (total 246 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Id                     1121 non-null   int32  
 1   MSSubClass             1121 non-null   int32  
 2   LotFrontage            1121 non-null   float64
 3   LotArea                1121 non-null   int32  
 4   OverallQual            1121 non-null   int32  
 5   OverallCond            1121 non-null   int32  
 6   YearBuilt              1121 non-null   int32  
 7   YearRemodAdd           1121 non-null   int32  
 8   MasVnrArea             1121 non-null   float64
 9   BsmtFinSF1             1121 non-null   int32  
 10  BsmtFinSF2             1121 non-null   int32  
 11  BsmtUnfSF              1121 non-null   int32  
 12  TotalBsmtSF            1121 non-null   int32  
 13  1stFlrSF               1121 non-null   int32  
 14  2ndFlrSF               1121 non-null   int32  
 15  Low